```
--- Day 12: Rain Risk ---
Your ferry made decent progress toward the island, but the storm came in faster than anyone expected. The ferry needs to take evasive actions!

Unfortunately, the ship's navigation computer seems to be malfunctioning; rather than giving a route directly to safety, it produced extremely circuitous instructions. When the captain uses the PA system to ask if anyone can help, you quickly volunteer.

The navigation instructions (your puzzle input) consists of a sequence of single-character actions paired with integer input values. After staring at them for a few minutes, you work out what they probably mean:

Action N means to move north by the given value.
Action S means to move south by the given value.
Action E means to move east by the given value.
Action W means to move west by the given value.
Action L means to turn left the given number of degrees.
Action R means to turn right the given number of degrees.
Action F means to move forward by the given value in the direction the ship is currently facing.
The ship starts by facing east. Only the L and R actions change the direction the ship is facing. (That is, if the ship is facing east and the next instruction is N10, the ship would move north 10 units, but would still move east if the following action were F.)

For example:

F10
N3
F7
R90
F11
These instructions would be handled as follows:

F10 would move the ship 10 units east (because the ship starts by facing east) to east 10, north 0.
N3 would move the ship 3 units north to east 10, north 3.
F7 would move the ship another 7 units east (because the ship is still facing east) to east 17, north 3.
R90 would cause the ship to turn right by 90 degrees and face south; it remains at east 17, north 3.
F11 would move the ship 11 units south to east 17, south 8.
At the end of these instructions, the ship's Manhattan distance (sum of the absolute values of its east/west position and its north/south position) from its starting position is 17 + 8 = 25.

Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?
```

In [1]:
#!fsharp
#r "nuget: Fparsec"
open FParsec

let input = System.IO.File.ReadAllLines "./input.txt"

type Direction =
    | North
    | East
    | South
    | West

type ShipState = { x : int; y : int; facing: int}

type Instruction =
    | MoveTowards of Direction * int
    | MoveForward of int
    | Turn of int

let instructionBuilder (action: char) (number: int) =
    match action with
    | 'N' -> MoveTowards (North, number)
    | 'S' -> MoveTowards (South, number)
    | 'E' -> MoveTowards (East, number)
    | 'W' -> MoveTowards (West, number)
    | 'L' -> Turn (-number)
    | 'R' -> Turn number
    | 'F' -> MoveForward number
    | _ -> failwith "Unknown instruction"

let parseInput : Parser<Instruction, unit> =
    pipe2 asciiUpper pint32 instructionBuilder

let instructionList = 
    Array.map (run parseInput) input 
    |> List.ofArray
    |> List.choose (function
        | Success (a, _, _) -> Some a
        | Failure _ -> None)

let rec direction (heading: int) =
    match (heading % 360) with
    | x when x < 0 -> direction (x + 360)
    | 0 -> North
    | 90 -> East
    | 180 -> South
    | 270 -> West
    | _ -> failwith $"Unknown direction of {heading}"

let rec execute (instruction: Instruction) (state: ShipState) : ShipState =
    match instruction with
    | MoveTowards (North, a) -> { state with y = state.y + a }
    | MoveTowards (East, a) -> { state with x = state.x + a }
    | MoveTowards (South, a) -> { state with y = state.y - a }
    | MoveTowards (West, a) -> { state with x = state.x - a }
    | MoveForward a -> execute (MoveTowards (direction state.facing, a)) state
    | Turn a -> {state with facing = ( (state.facing + a) % 360 )}

let finalState = List.fold (fun state ins -> execute ins state) { x = 0 ; y = 0; facing = 90 } instructionList

display finalState
display (abs (finalState.x) + abs (finalState.y))

x,y,facing
-589,632,-270


1221

```
--- Part Two ---
Before you can give the destination to the captain, you realize that the actual action meanings were printed on the back of the instructions the whole time.

Almost all of the actions indicate how to move a waypoint which is relative to the ship's position:

Action N means to move the waypoint north by the given value.
Action S means to move the waypoint south by the given value.
Action E means to move the waypoint east by the given value.
Action W means to move the waypoint west by the given value.
Action L means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees.
Action R means to rotate the waypoint around the ship right (clockwise) the given number of degrees.
Action F means to move forward to the waypoint a number of times equal to the given value.
The waypoint starts 10 units east and 1 unit north relative to the ship. The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.

For example, using the same instructions as above:

F10 moves the ship to the waypoint 10 times (a total of 100 units east and 10 units north), leaving the ship at east 100, north 10. The waypoint stays 10 units east and 1 unit north of the ship.
N3 moves the waypoint 3 units north to 10 units east and 4 units north of the ship. The ship remains at east 100, north 10.
F7 moves the ship to the waypoint 7 times (a total of 70 units east and 28 units north), leaving the ship at east 170, north 38. The waypoint stays 10 units east and 4 units north of the ship.
R90 rotates the waypoint around the ship clockwise 90 degrees, moving it to 4 units east and 10 units south of the ship. The ship remains at east 170, north 38.
F11 moves the ship to the waypoint 11 times (a total of 44 units east and 110 units south), leaving the ship at east 214, south 72. The waypoint stays 4 units east and 10 units south of the ship.
After these operations, the ship's Manhattan distance from its starting position is 214 + 72 = 286.

Figure out where the navigation instructions actually lead. What is the Manhattan distance between that location and the ship's starting position?
```


In [1]:
#!fsharp
type State = { shipCoord: int * int ; wpCoord : int * int }


let rec rotateCoordinates (c: int * int) (rotation: int) =
    let rotate90 (x, y) = (-y, x)
    let rotate180 (x, y) = (-x, -y)
    let rotate270 (x, y) = (y, -x)

    match (rotation) with
    | x when x < 0 -> rotateCoordinates c (rotation + 360)
    | 0 -> c
    | 90 -> rotate90 c
    | 180 -> rotate180 c
    | 270 -> rotate270 c
    | x -> failwith $"Unknown rotation of {rotation}"

let execute2 (instruction: Instruction) (state: State) =
    let (shipX, shipY) = state.shipCoord
    let (wpX, wpY) = state.wpCoord
    match instruction with
    | MoveTowards (North, a) -> { state with wpCoord = (wpX, wpY + a) }
    | MoveTowards (East, a) -> { state with wpCoord = (wpX + a, wpY) }
    | MoveTowards (South, a) -> { state with wpCoord = (wpX, wpY - a) }
    | MoveTowards (West, a) -> { state with wpCoord = (wpX - a, wpY) }
    | MoveForward a -> {state with shipCoord = (shipX + (wpX * a), shipY + (wpY * a))}
    | Turn a -> {state with wpCoord = rotateCoordinates state.wpCoord (-a) }

let finalState2 = List.fold (fun state ins -> execute2 ins state) { shipCoord = (0, 0); wpCoord = (10, 1)} instructionList


display finalState2
display (abs (finalState2.shipCoord |> fst) + abs (finalState2.shipCoord |> snd))

shipCoord,wpCoord
"( -32313, 27122 )","( 50, -28 )"


59435